In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import pathlib
from datetime import datetime
import math
import sys

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates

from sklearn import metrics
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

import xgboost as xg

%matplotlib inline

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)




In [2]:
df = pd.read_csv('/Users/yashwanthkaruparthi/Developer/energy_demand/research/data/7-usable/1-daily_data.csv')

df_clean = pd.read_csv('/Users/yashwanthkaruparthi/Developer/energy_demand/research/clean-preprocessed_last.csv')

In [3]:
df_clean.head()

,Unnamed: 0,temp_min,temp_max,temp_mean,temp_median,temp_max_hour,temp_min_hour,dew_point_temp_min,dew_point_temp_max,dew_point_temp_mean,dew_point_temp_median,dew_point_temp_max_hour,dew_point_temp_min_hour,rel_hum_min,rel_hum_max,rel_hum_mean,rel_hum_median,rel_hum_max_hour,rel_hum_min_hour,visibility_min,visibility_max,visibility_mean,visibility_median,visibility_max_hour,visibility_min_hour,press_min,press_max,press_mean,press_median,press_max_hour,press_min_hour,hmdxx_min,hmdxx_max,hmdxx_mean,hmdxx_median,hmdxx_max_hour,hmdxx_min_hour,sun_set,sun_rise,hourly_demand,day_of_week,week_of_year,year
0,1994-01-01,-1.8,2.8,0.575000,0.65,12.0,5.0,-4.8,1.1,-1.387500,-0.65,19.0,7.0,73.0,99.0,87.000000,86.5,14.0,9.0,0.8,40.2,17.841667,16.1,11.0,14.0,98.51,99.91,99.075833,98.840,0.0,16.0,-4.976663,0.686009,-1.884900,-1.513575,13.0,7.0,17.0,8.0,14152.0,5.0,52.0,1994.0
1,1994-01-02,-14.3,1.7,-8.704167,-9.90,0.0,23.0,-20.3,0.5,-12.445833,-13.90,0.0,23.0,60.0,92.0,74.416667,75.5,0.0,23.0,6.4,40.2,28.387500,25.0,9.0,6.0,98.77,100.51,99.920000,100.125,18.0,0.0,-19.165499,-0.338394,-12.827363,-14.393998,0.0,23.0,17.0,8.0,16527.0,6.0,52.0,1994.0
2,1994-01-03,-16.3,-10.3,-12.612500,-12.00,17.0,2.0,-23.2,-12.6,-15.820833,-15.00,17.0,2.0,55.0,91.0,77.250000,80.5,9.0,2.0,1.2,25.0,10.404167,8.0,0.0,9.0,99.41,100.47,100.057083,100.155,0.0,23.0,-21.317386,-14.555717,-17.143932,-16.423198,17.0,2.0,17.0,8.0,17550.0,0.0,1.0,1994.0
3,1994-01-04,-11.5,-7.4,-9.833333,-9.95,15.0,8.0,-15.0,-11.5,-13.570833,-13.70,13.0,0.0,57.0,84.0,74.416667,76.0,10.0,16.0,9.7,40.2,24.879167,24.1,13.0,10.0,98.27,99.34,98.675417,98.610,0.0,14.0,-15.874948,-11.706660,-14.183286,-14.310177,15.0,8.0,17.0,8.0,17395.0,1.0,1.0,1994.0
4,1994-01-05,-17.5,-7.2,-11.866667,-11.75,5.0,22.0,-20.8,-10.7,-16.154167,-17.00,4.0,22.0,56.0,84.0,70.791667,74.0,4.0,14.0,1.0,40.2,27.033333,25.0,9.0,7.0,98.44,100.37,99.352083,99.375,22.0,3.0,-22.394182,-11.256395,-16.407695,-16.464188,5.0,22.0,17.0,8.0,18485.0,2.0,1.0,1994.0


In [4]:
df_clean.rename(columns = {'Unnamed: 0':'date'}, inplace = True)
df_clean = df_clean.drop(['year'],axis=1)
df_clean = df_clean.astype({'week_of_year':'string'})
df_clean = df_clean.astype({'day_of_week':'string'})
df_clean['date'] = pd.to_datetime(df_clean['date'])
df_clean['year'] = df_clean['date'].dt.year
df_clean['day'] = df_clean['date'].dt.day
df_clean['month'] = df_clean['date'].dt.month
df_clean = df_clean.drop(['date','sun_rise'],axis=1)
print(df_clean)

      temp_min  temp_max  temp_mean  temp_median  temp_max_hour  temp_min_hour  dew_point_temp_min  dew_point_temp_max  dew_point_temp_mean  dew_point_temp_median  dew_point_temp_max_hour  dew_point_temp_min_hour  rel_hum_min  rel_hum_max  rel_hum_mean  rel_hum_median  rel_hum_max_hour  rel_hum_min_hour  visibility_min  visibility_max  visibility_mean  visibility_median  visibility_max_hour  visibility_min_hour  press_min  press_max  press_mean  press_median  press_max_hour  press_min_hour  hmdxx_min  hmdxx_max  hmdxx_mean  hmdxx_median  hmdxx_max_hour  hmdxx_min_hour  sun_set  hourly_demand day_of_week week_of_year  year  day  month
0         -1.8       2.8   0.575000         0.65           12.0            5.0                -4.8                 1.1            -1.387500                  -0.65                     19.0                      7.0         73.0         99.0     87.000000            86.5              14.0               9.0             0.8            40.2        17.841667     

In [5]:
# df.head()

In [6]:
df_clean.rename(columns={'hourly_demand':'daily_demand'}, inplace=True)

In [7]:
# df['date'] = pd.to_datetime(df['date'])

In [8]:
# df.set_index('date', inplace=True)

In [9]:
# df['year'] = df.index.year
# df['month'] = df.index.month
# df['day'] = df.index.day
# df['day_of_week'] = df.index.dayofweek
# df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
# df['week_of_year'] = df.index.isocalendar().week

In [10]:
# df.head()

In [11]:
# df.shape

### Generating Lag features by one day and 7 day period

In [12]:
df_clean['demand_lag_1'] = df_clean['daily_demand'].shift(1)
df_clean['demand_lag_7'] = df_clean['daily_demand'].shift(7)


In [13]:
# df.head(30)

In [14]:
# df.dropna(subset=['demand_lag_1', 'demand_lag_7'], inplace = True)

In [15]:
# df.head(30)

In [16]:
# print(df.columns)

In [17]:
# df.drop(columns=['temp', 'dew_point_temp', 'rel_hum', 'visibility','press', 'hmdxx'], inplace = True)

In [18]:
# df = df.reset_index()

In [19]:
# df.drop(columns=['date'], inplace=True)

In [26]:
df_clean.head()

,temp_min,temp_max,temp_mean,temp_median,temp_max_hour,temp_min_hour,dew_point_temp_min,dew_point_temp_max,dew_point_temp_mean,dew_point_temp_median,dew_point_temp_max_hour,dew_point_temp_min_hour,rel_hum_min,rel_hum_max,rel_hum_mean,rel_hum_median,rel_hum_max_hour,rel_hum_min_hour,visibility_min,visibility_max,visibility_mean,visibility_median,visibility_max_hour,visibility_min_hour,press_min,press_max,press_mean,press_median,press_max_hour,press_min_hour,hmdxx_min,hmdxx_max,hmdxx_mean,hmdxx_median,hmdxx_max_hour,hmdxx_min_hour,sun_set,daily_demand,day_of_week,week_of_year,year,day,month,demand_lag_1,demand_lag_7
0,-1.8,2.8,0.575000,0.65,12.0,5.0,-4.8,1.1,-1.387500,-0.65,19.0,7.0,73.0,99.0,87.000000,86.5,14.0,9.0,0.8,40.2,17.841667,16.1,11.0,14.0,98.51,99.91,99.075833,98.840,0.0,16.0,-4.976663,0.686009,-1.884900,-1.513575,13.0,7.0,17.0,14152.0,5.0,52.0,1994,1,1,NaN,NaN
1,-14.3,1.7,-8.704167,-9.90,0.0,23.0,-20.3,0.5,-12.445833,-13.90,0.0,23.0,60.0,92.0,74.416667,75.5,0.0,23.0,6.4,40.2,28.387500,25.0,9.0,6.0,98.77,100.51,99.920000,100.125,18.0,0.0,-19.165499,-0.338394,-12.827363,-14.393998,0.0,23.0,17.0,16527.0,6.0,52.0,1994,2,1,14152.0,NaN
2,-16.3,-10.3,-12.612500,-12.00,17.0,2.0,-23.2,-12.6,-15.820833,-15.00,17.0,2.0,55.0,91.0,77.250000,80.5,9.0,2.0,1.2,25.0,10.404167,8.0,0.0,9.0,99.41,100.47,100.057083,100.155,0.0,23.0,-21.317386,-14.555717,-17.143932,-16.423198,17.0,2.0,17.0,17550.0,0.0,1.0,1994,3,1,16527.0,NaN
3,-11.5,-7.4,-9.833333,-9.95,15.0,8.0,-15.0,-11.5,-13.570833,-13.70,13.0,0.0,57.0,84.0,74.416667,76.0,10.0,16.0,9.7,40.2,24.879167,24.1,13.0,10.0,98.27,99.34,98.675417,98.610,0.0,14.0,-15.874948,-11.706660,-14.183286,-14.310177,15.0,8.0,17.0,17395.0,1.0,1.0,1994,4,1,17550.0,NaN
4,-17.5,-7.2,-11.866667,-11.75,5.0,22.0,-20.8,-10.7,-16.154167,-17.00,4.0,22.0,56.0,84.0,70.791667,74.0,4.0,14.0,1.0,40.2,27.033333,25.0,9.0,7.0,98.44,100.37,99.352083,99.375,22.0,3.0,-22.394182,-11.256395,-16.407695,-16.464188,5.0,22.0,17.0,18485.0,2.0,1.0,1994,5,1,17395.0,NaN


In [21]:
# df.drop(columns=['is_weekend'], inplace=True)


In [22]:
x = df_clean.loc[:, df_clean.columns != 'daily_demand'].values
y = df_clean.loc[:, 'daily_demand'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [23]:
xgb_r = xg.XGBRegressor(objective ='reg:squarederror', n_estimators = 10, seed = 123)

xgb_r.fit(x_train, y_train) 

pred = xgb_r.predict(x_test) 

In [24]:
r_sq = metrics.r2_score(y_test,pred)

print(r_sq)

0.9413614324921541


In [25]:
# df_clean.to_csv('/Users/yashwanthkaruparthi/Developer/energy_demand/research/data/8-feat-eng/feat-lag.csv')